In [1]:
# Projeto Integrador VI
# Engenharia da Computação
# Pólo Americana
# Fabio Wakisaka
# 1800221

from kivy.app import App
from kivy.uix.gridlayout import GridLayout
from kivy.uix.label import Label
from kivy.uix.image import Image
from kivy.uix.button import Button
from kivy.uix.textinput import TextInput

import kivy
from kivymd.app import MDApp
from kivymd.uix.boxlayout import MDBoxLayout
from kivymd.uix.button import MDRaisedButton
from kivy.uix.image import Image
from kivy.graphics.texture import Texture
from kivy.clock import Clock

from kivy.uix.widget import Widget
from kivy.lang import Builder
from kivy.uix.screenmanager import ScreenManager, Screen

import cv2

import requests
import json
import random
import numpy as np

link = "https://projeto-integrador-vi-93572-default-rtdb.firebaseio.com/"
kivy.require('1.9.0')


import firebase_admin
from firebase_admin import credentials, storage

from datetime import datetime, timedelta
ini_time_for_now = str(datetime.now())
 

cred = credentials.Certificate("projeto-integrador-vi-93572-firebase-adminsdk-g5xbs-216fa6d5aa.json")
firebase_admin.initialize_app(cred,{'storageBucket': 'projeto-integrador-vi-93572.appspot.com'})


class App(App):
    def build(self):
        layout = MDBoxLayout(orientation = 'vertical')

        layout = GridLayout()
        layout.cols = 1
        layout.size_hint = (0.6, 0.7)
        layout.pos_hint = {"center_x": 0.5, "center_y":0.5}
        
        self.header = Label(
                            text="Wiifit ",
                            font_size = 28,
                            color="#00FFCE")
        layout.add_widget(self.header)
        
        self.image = Image()
        layout.add_widget(self.image)

        self.capture = cv2.VideoCapture(0)
        Clock.schedule_interval(self.load_video, 1.0/30.0)        
        
        self.greeting = Label(
                            text="Olá, qual seu nome? ",
                            font_size = 18,
                            color="#00FFCE")
        layout.add_widget(self.greeting)

        self.user = TextInput(
                    multiline=False,
                    padding_y = (20,20),
                    size_hint = (1, 0.7)
                    )
        layout.add_widget(self.user)
        
        self.button = Button(
                        text="Enviar foto",
                        size_hint = (1,0.5),
                        bold = True,
                        background_color = "#00FFCE"
                        )
        self.button.bind(on_press=self.take_picture)
        layout.add_widget(self.button)
        
        self.buttonget = Button(
                        text="Buscar foto",
                        size_hint = (1,0.5),
                        bold = True,
                        background_color = "#00FFFF"
                        )
        self.buttonget.bind(on_press=self.create_get)
        layout.add_widget(self.buttonget)        
        return layout
    
    def load_video(self, *args):
        ret, frame = self.capture.read()
        self.image_frame = frame
        buffer = cv2.flip(frame, 0).tostring()
        texture = Texture.create(size=(frame.shape[1], frame.shape[0]), colorfmt = 'bgr')
        texture.blit_buffer(buffer, colorfmt = 'bgr', bufferfmt = 'ubyte')
        self.image.texture = texture

    
    def take_picture(self, *args):
        self.greeting.text = "Olá " + self.user.text + ", foto enviada!"
        image_name = self.user.text + ".png"
        cv2.imwrite(image_name, self.image_frame)
        file_path = image_name
        bucket = storage.bucket()
        blob = bucket.blob(file_path)
        blob.upload_from_filename(file_path)
        dados = {'nome': self.user.text, 'hashtag': image_name, 'datetime': ini_time_for_now}
        requisicao = requests.post(f'{link}/Usuarios/.json', data=json.dumps(dados))

        
    def create_get(self, *args):
        print("Visualização de foto ")
        res = requests.get(f'{link}/Usuarios/.json')
        self.greeting.text = "Última foto enviada em nova janela"
        bucket = storage.bucket()
        blob = bucket.get_blob("fabio.png")
        arr = np.frombuffer(blob.download_as_string(), np.uint8)
        img = cv2.imdecode(arr, cv2.COLOR_BGR2BGR555)
        self.image2 = cv2.imshow('image', img)
        
if __name__ == "__main__":
    App().run()
    
    
    

[INFO   ] [Logger      ] Record log in C:\Users\vani_\.kivy\logs\kivy_22-09-17_6.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.3.2
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.1
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.4.5
[INFO   ] [Kivy        ] v2.1.0
[INFO   ] [Kivy        ] Installed at "c:\users\vani_\appdata\local\programs\python\python39\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "c:\users\vani_\appdata\local\programs\python\python39\python.exe"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 189 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL

Visualização de foto 


[INFO   ] [Base        ] Leaving application in progress...
